In [195]:
import pandas as pd
import numpy as np
from datetime import datetime


In [196]:
all_seasons = pd.read_csv('all_seasons.csv')
all_seasons = all_seasons[['datetime','conditions']]
all_seasons.head()


,datetime,conditions
0,1/1/2000,Partially cloudy
1,2/1/2000,Clear
2,3/1/2000,Clear
3,4/1/2000,Clear
4,5/1/2000,Clear


In [197]:
simplifier = {'Overcast':'cloudy','Partially cloudy':'cloudy','Clear':'sunny','Rain':'rainy','Rain,Partially cloudy':'rainy','Rain,Overcast':'rainy'}
all_seasons['condition']= all_seasons['conditions'].map(simplifier)
all_seasons.head()

,datetime,conditions,condition
0,1/1/2000,Partially cloudy,cloudy
1,2/1/2000,Clear,sunny
2,3/1/2000,Clear,sunny
3,4/1/2000,Clear,sunny
4,5/1/2000,Clear,sunny


In [198]:
all_seasons = all_seasons[['datetime','condition']]
all_seasons.head()

,datetime,condition
0,1/1/2000,cloudy
1,2/1/2000,sunny
2,3/1/2000,sunny
3,4/1/2000,sunny
4,5/1/2000,sunny


In [199]:
train_start_date = '1/1/2000'
train_end_date = '1/1/2022'
train_start_date = pd.to_datetime(train_start_date, format='%d/%m/%Y')
train_end_date = pd.to_datetime(train_end_date, format='%d/%m/%Y')
all_seasons['datetime'] = pd.to_datetime(all_seasons['datetime'], format='%d/%m/%Y')
all_seasons_train = all_seasons.loc[(all_seasons['datetime'] >= train_start_date) & (all_seasons['datetime'] <= train_end_date)]
# # เรียงข้อมูลตามวันที่
all_seasons_train = all_seasons_train.sort_values(by='datetime')

# # รีเซ็ตดัชนี
all_seasons_train = all_seasons_train.reset_index(drop=True)



all_seasons_train


,datetime,condition
0,2000-01-01,cloudy
1,2000-01-02,sunny
2,2000-01-03,sunny
3,2000-01-04,sunny
4,2000-01-05,sunny
...,...,...
8032,2021-12-28,NaN
8033,2021-12-29,NaN
8034,2021-12-30,NaN
8035,2021-12-31,cloudy


In [200]:
#set conditions count S=Sunny R=Rain C=cloudy 
S_after_S_count=0.0
S_after_R_count=0.0
S_after_C_count=0.0

C_after_S_count=0.0
C_after_R_count=0.0
C_after_C_count=0.0

R_after_S_count=0.0
R_after_R_count=0.0
R_after_C_count=0.0

In [201]:
#count
all_seasons_train['condition_shift'] = all_seasons_train['condition'].shift(-1)

for i in range (len(all_seasons_train)):
    if all_seasons_train.loc[i, 'condition'] == 'sunny' and all_seasons_train.loc[i,'condition_shift'] == 'sunny': 
        S_after_S_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'sunny' and all_seasons_train.loc[i,'condition_shift'] == 'rainy': 
        S_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'sunny' and all_seasons_train.loc[i,'condition_shift'] == 'cloudy': 
        S_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rainy' and all_seasons_train.loc[i,'condition_shift'] == 'rainy': 
        R_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rainy' and all_seasons_train.loc[i,'condition_shift'] == 'sunny': 
        R_after_S_count += 1 
    elif all_seasons_train.loc[i, 'condition'] == 'rainy' and all_seasons_train.loc[i,'condition_shift'] == 'cloudy': 
        R_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'cloudy' and all_seasons_train.loc[i,'condition_shift'] == 'rainy': 
        C_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'cloudy' and all_seasons_train.loc[i,'condition_shift'] == 'sunny': 
        C_after_S_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'cloudy' and all_seasons_train.loc[i,'condition_shift'] == 'cloudy': 
        C_after_C_count += 1

In [202]:
current_S_total = S_after_S_count + S_after_R_count + S_after_C_count
current_R_total = R_after_S_count + R_after_R_count + R_after_C_count
current_C_total = C_after_S_count + C_after_R_count + C_after_C_count
print(current_C_total)


3159.0


In [203]:
S_after_S_prob = S_after_S_count / current_S_total
S_after_R_prob = S_after_R_count / current_S_total 
S_after_C_prob = S_after_C_count / current_S_total

R_after_R_prob = R_after_R_count/current_R_total
R_after_S_prob = R_after_S_count/current_R_total
R_after_C_prob = R_after_C_count/current_R_total

C_after_C_prob = C_after_C_count/current_C_total
C_after_S_prob = C_after_S_count/current_C_total
C_after_R_prob = C_after_R_count/current_C_total



In [204]:
print(S_after_S_count)
print(S_after_C_count)
print(S_after_R_count)

2328.0
830.0
56.0


In [206]:
print(S_after_S_prob)
print(S_after_R_prob)
print(S_after_C_prob)
print(R_after_S_prob)
print(R_after_R_prob)
print(R_after_C_prob)
print(C_after_S_prob)
print(C_after_R_prob)
print(C_after_C_prob)
print(R_after_C_prob+R_after_R_prob+R_after_S_prob)
print(S_after_C_prob+S_after_R_prob+S_after_S_prob)
print(C_after_C_prob+)

0.7243310516490354
0.017423771001866834
0.2582451773490977
0.6691176470588235
0.15441176470588236
0.17647058823529413
0.2241215574548908
0.011079455523899969
0.7647989870212093
1.0
